In [27]:
%pip install langchain text-generation transformers runpod python-dotenv ipython chromadb torch einops

     ---------------------------------------- 0.0/42.2 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.2 kB ? eta -:--:--
     ------------------------------------ - 41.0/42.2 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 42.2/42.2 kB 410.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [36]:
DeviceAliasList = [ "Livingroom.light",
"Bedroom.light",
"Bedroom.tv",
"Kitchen.light",
"Bathroom.light",
"Study.light",
"Garage.light",
"Patio.light",
"Dining.light",
"Guestroom.light",
"Master.light",
"Basement.light",
"Office.light",
"Kidsroom.light",
"Hallway.light",
"Balcony.light",
"Entryway.light",
"Livingroom.table_light",
"Entertainment.light"]

In [4]:
DeviceAliasList = [ "Livingroom Lights: Livingroom.light",
"Bedroom Lights: Bedroom.light",
"TV: Bedroom.tv",
"Kitchen Lights: Kitchen.light",
"Bathroom Lights: Bathroom.light",
"Study Lights: Study.light",
"Garage Lights: Garage.light",
"Patio Lights: Patio.light",
"Dining Lights: Dining.light",
"Guestroom Lights: Guestroom.light",
"Master Lights: Master.light",
"Basement Lights: Basement.light",
"Office Lights: Office.light",
"Kidsroom Lights: Kidsroom.light",
"Hallway Lights: Hallway.light",
"Balcony Lights: Balcony.light",
"Entryway Lights: Entryway.light",
"Entertainment Lights: Entertainment.light"]

In [6]:
import chromadb
client = chromadb.Client()
#client.delete_collection("devices")

collection = client.create_collection("devices")

collection.add(
    documents=DeviceAliasList,
    ids=[str(i) for i in range(0, len(DeviceAliasList))]
)

In [7]:
from langchain.llms import HuggingFaceTextGenInference

inference_server_url = f'http://localhost:8080'
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=100,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.1,
)

c:\Users\jonas\.conda\envs\Masterarbeit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from IPython.display import display, Markdown
from langchain import PromptTemplate
from langchain.chains import LLMChain

Large Promt:

In [29]:
prompt = PromptTemplate(
    input_variables=["User_request"],
    template="""
### Instruction:    
Below you will find a "USER REQUEST" to a Smart Home System. The USER REQUEST need to be converted into the following Format: "COMMAND: [device name]: [action]". Replace "device name" with the alias from the list below. Also replace "[action]" with the fitting selection from the action list. You may also issue multiple commands if you nessesary.

Device Alias list:
Livingroom Lights: "Livingroom.light"
Bedroom Lights: "Bedroom.light"
TV: "livingroom.tv"
Kitchen Lights: "Kitchen.light"
Bathroom Lights: "Bathroom.light"
Study Lights: "Study.light"
Garage Lights: "Garage.light"
Patio Lights: "Patio.light"
Dining Lights: "Dining.light"
Guestroom Lights: "Guestroom.light"
Master Lights: "Master.light"
Basement Lights: "Basement.light"
Office Lights: "Office.light"
Kidsroom Lights: "Kidsroom.light"
Hallway Lights: "Hallway.light"
Balcony Lights: "Balcony.light"
Entryway Lights: "Entryway.light"
Entertainment Lights: "Entertainment.light"

Action list:
on
off
dim x%
color x

USER REQUEST: set the lights in the office to 50%
RESPONSE: COMMAND: Office.light: set 50%
USER REQUEST: Turn on the light in the Hallway!
RESPONSE: COMMAND: Hallway.light: on
USER REQUEST: Turn on the light in the Bathroom and kitchen
RESPONSE: COMMAND: Office.light: on \n COMMAND: kitchen.light: on
USER REQUEST: {User_request}
### RESPONSE: 
"""
)

chain = LLMChain(llm=llm, prompt=prompt)

USER_REQUEST:

In [32]:
User_request = "Are the lights in the kitchen on?"
json_response_template_router = """{{"destination": string, "next_inputs": string}}"""
destination_options = """Command: If the user is trying to change the state of a device.\nInquiry: If the user is requesting information about a device"""

Router:

In [10]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [54]:
router_template = """
### Instruction:    
Below are some "USER REQUESTS" and a "RESPONSE" from a Smart Home System. Respond in a JSON Markdown format as defined under "JSON STRUCTURE". Change the "destination" to one of the options provided under "DESTINATION OPTIONS". 

JSON STRUCTURE: {json_response_template}

DESTINATION OPTIONS:
{destination_options}

USER REQUEST: "set the lights in the office to 50%"
RESPONSE: RESPONSE: {{"destination": "Command", "next_inputs": "set the lights in the office to 50%"}}

USER REQUEST: "Is the light in the Kitchen on?"
RESPONSE: {{"destination": "Inquiry", "next_inputs": "Is the light in the Kitchen on?"}}

USER REQUEST: "Turn on the light in the livingroom!"
RESPONSE: {{"destination": "Command", "next_inputs": "Turn on the light in the livingroom!"}}

USER REQUEST: "{input}"
### RESPONSE:
"""



commandMocPrompt = """Tell me the name of an Animal: """

inquiryMocPrompt = """1, 2, 3, 4, 5, 6, 7, 8, """

prompt_infos = [
    {
        "name": "Command",
        "description": "Used if the user wants to change the state of a device",
        "prompt_template": commandMocPrompt,
    },
    {
        "name": "Inquiry",
        "description": "Used if the user wants to know information about the device.",
        "prompt_template": inquiryMocPrompt,
    },
]

destination_chains = {}
for info in prompt_infos:
    name = info["name"]
    prompt_template = info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=[])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

chain = LLMChain(llm=llm, prompt=prompt)

In [43]:
import json
from langchain.schema import BaseOutputParser, OutputParserException
from typing import Dict, Any, Type

class CustomOutputParser(BaseOutputParser[Dict[str, str]]):
    """Customized Json output Parser"""

    default_destination: str = "DEFAULT"
    next_inputs_type: Type = str
    next_inputs_inner_key: str = "input"

    def parse(self, text: str) -> Dict[str, Any]:
        try:
            parsed = json.loads(text)
            if not isinstance(parsed["destination"], str):
                raise ValueError("Expected 'destination' to be a string.")
            if not isinstance(parsed["next_inputs"], self.next_inputs_type):
                raise ValueError(
                    f"Expected 'next_inputs' to be {self.next_inputs_type}."
                )
            parsed["next_inputs"] = {self.next_inputs_inner_key: parsed["next_inputs"]}
            if (
                parsed["destination"].strip().lower()
                == self.default_destination.lower()
            ):
                parsed["destination"] = None
            else:
                parsed["destination"] = parsed["destination"].strip()
            return parsed
        except Exception as e:
            raise OutputParserException(
                f"Parsing text\n{text}\n raised following error:\n{e}"
            )


In [45]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [51]:
#destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
#destinations_str = "\n".join(destinations)
#router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input", "json_response_template", "destination_options"],
    output_parser=CustomOutputParser(),
    verbose=True,
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [35]:
prompt = PromptTemplate(
    input_variables=["input", "json_response_template", "destination_options"],
    template=router_template)
chain = LLMChain(llm=llm, prompt=prompt)

In [53]:
llmOut = chain.run({"input": "Turn on the light in the Kitchen!", "json_response_template": json_response_template_router, "destination_options": destination_options})
display(Markdown(llmOut))



> Entering new MultiPromptChain chain...
Command: {'input': 'Turn on the light in the Kitchen!'}
> Finished chain.


100%
The answer to this question is a simple one. The animal you are thinking of is a dog. Dogs have been around for thousands of years and they are still very popular today. They come in all shapes and sizes, but most people think of them as being small or medium sized animals with fur on their bodies.
What do you call a group of dogs?
A pack of dogs is called a kennel.
How many dogs can be in a

Command identification:

In [55]:
json_response_template_command = """[{{"DEVICE": string,
"ACTION": string}}]"""

In [56]:
prompt = PromptTemplate(
    input_variables=["User_request", "device_list", "json_response_template_command"],
    template="""
### Instruction:    
Below are "USER REQUESTS" and a "RESPONSE" from a Smart Home System. Respond in a JSON array format as defined under "JSON STRUCTURE". Choose a fitting "DEVICE" from the "DEVICE LIST" and "ACTIOn" from the "ACTION LIST". If multiple actions are required, list all devices and actions.

YAML STRUCTURE: {json_response_template_command}

ACTION LIST: turn on, turn off, dim x, color x

DEVICE LIST: {device_list}

USER REQUEST: set the lights in the office to 50%
RESPONSE: [{{"DEVICE": Office.light,
"ACTION": dim 50}}]
USER REQUEST: Turn on the light in the Hallway!
RESPONSE: [{{"DEVICE": Hallway.light,
"ACTION": turn on}}]
USER REQUEST: turn off the light in the Kitchen
RESPONSE: [{{"DEVICE": Kitchen.light,
"ACTION": turn off}}]
USER REQUEST: Change the color in the kitchen to white
RESPONSE: [{{"DEVICE": Kitchen.light,
"ACTION": color white}}]
USER REQUEST: Change the color in the kitchen to white and turn on the light in the bathroom
RESPONSE: [{{"DEVICE": Kitchen.light,
"ACTION": color white}},
{{"DEVICE": Bathroom.light,
"ACTION": turn on}}]
USER REQUEST: {User_request}
### RESPONSE: 
"""
)

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
prompt = PromptTemplate(
    input_variables=["User_request", "device_list", "json_response_template_command"],
    template="""
### Instruction:    
Below are "USER REQUESTS" and a "RESPONSE" from a Smart Home System. Respond in a JSON array format as defined under "JSON STRUCTURE". Choose a "DEVICE" from the "DEVICE LIST" and "ACTION" from the "ACTION LIST". If multiple actions are required, list all devices and actions.

JSON STRUCTURE: [{{"DEVICE": string, "ACTION": string}}]

ACTION LIST: turn on, turn off, dim x, color x

DEVICE LIST: 
{device_list}

REMEMBER: Only list multiple devices if absolutely necessary

USER REQUEST: set the lights in the office to 50%
RESPONSE: [{{"DEVICE": Office.light,
"ACTION": dim 50}}]
USER REQUEST: Turn on the light in the Hallway!
RESPONSE: [{{"DEVICE": Hallway.light,
"ACTION": turn on}}]
USER REQUEST: turn off the light in the Kitchen
RESPONSE: [{{"DEVICE": Kitchen.light,
"ACTION": turn off}}]
USER REQUEST: Change the color in the kitchen to white
RESPONSE: [{{"DEVICE": Kitchen.light,
"ACTION": color white}}]
USER REQUEST: Change the color in the kitchen to white and turn on the light in the bathroom
RESPONSE: [{{"DEVICE": Kitchen.light,
"ACTION": color white}},
{{"DEVICE": Bathroom.light,
"ACTION": turn on}}]

USER REQUEST:
### RESPONSE: 
"""
)

chain = LLMChain(llm=llm, prompt=prompt)


In [62]:
#overwrite User_request
#User_request = "Change the color in the kitchen to white and turn on the light in the bathroom. Oh and can you also turn off the TV?"
User_request = "turn on the TV"

In [63]:
vectorDBResults = collection.query(
    query_texts=[User_request],
    n_results=10)

print(vectorDBResults["documents"][0])

display(Markdown(chain.run({"User_request": User_request, "device_list": vectorDBResults["documents"][0], "json_response_template_command": json_response_template_command})))

['TV: Bedroom.tv', 'Entertainment Lights: Entertainment.light', 'Livingroom Lights: Livingroom.light', 'Bedroom Lights: Bedroom.light', 'Office Lights: Office.light', 'Dining Lights: Dining.light', 'Kitchen Lights: Kitchen.light', 'Guestroom Lights: Guestroom.light', 'Study Lights: Study.light', 'Kidsroom Lights: Kidsroom.light']


[{"DEVICE": Bedroom.tv,
"ACTION": turn on}]

In [60]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Turn on all devices in the livingroom!"],
    n_results=10,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    # where_document={"$contains":"search_string"}  # optional filter
)

results["documents"][0]

['TV: Bedroom.tv',
 'Livingroom Lights: Livingroom.light',
 'Bedroom Lights: Bedroom.light',
 'Kidsroom Lights: Kidsroom.light',
 'Office Lights: Office.light',
 'Kitchen Lights: Kitchen.light',
 'Guestroom Lights: Guestroom.light',
 'Bathroom Lights: Bathroom.light',
 'Study Lights: Study.light',
 'Hallway Lights: Hallway.light']

Type (request) -> room -> device/action

In [58]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

sysprompt = """Below is a \"USER\" asking to make a change to a smart home device. Respond in JSON format as defined under \"JSON STRUCTURE\". All available devices and their alias are defined in the \"DEVICE ALIAS LIST\" in the format \"Alias: device_id\", choose the best fitting device. Also choose an \"action\" from the \"ACTION LIST\". \n\n\nJSON STRUCTURE: {{\"type\": \"command\", \"commands\": [{{\"device\": device_id, \"action\": string, \"state\": string}}]}}\n\nACTION LIST: power [STATE: on, off], dim [STATE: percentage], color [STATE: color name]\n\nDEVICE LIST: \nbathroom light: light.bathroom,\nlivingroom light: light.livingroom,\nKitchen light: light.kitchen,\nbedroom light: light.bedroom,\ngarage light: light.garage,\nstudy room light: light.study_room\n\nUSER REQUEST: \"Turn on the Bathroom Light.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.bathroom\", \"action\": \"power\", \"state\": \"on\"}}]}}\n\nUSER REQUEST: \"Turn the Livingroom Light on.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.livingroom\", \"action\": \"power\", \"state\": \"on\"}}]}}\n\nUSER REQUEST: \"Turn off the Light in the Kitchen.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.kitchen\", \"action\": \"power\", \"state\": \"off\"}}]}}\n\nUSER REQUEST: \"Set the color in the livingroom to green.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.livingroom\", \"action\": \"color\", \"state\": \"green\"}}]}}\n\nUSER REQUEST: \"Enable the Livingroom Light.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [[{{\"device\": \"light.livingroom\", \"action\": \"power\", \"state\": \"on\"}}]}}\n\nUSER REQUEST: \"Dim the light in the kitchen to 50%.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.kitchen\", \"action\": \"dim\", \"state\": \"50\"}}]}}"""

model = AutoModelForCausalLM.from_pretrained("teknium/Puffin-Phi-v2", trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("teknium/Puffin-Phi-v2", trust_remote_code=True, torch_dtype=torch.bfloat16)
inputs = tokenizer(f"{sysprompt}\n\nUSER: Turn on the livingroom light.\nASSISTANT:", return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=128, do_sample=True, temperature=0.2, top_p=0.9, use_cache=True, repetition_penalty=1.2, eos_token_id=tokenizer.eos_token_id)
text = tokenizer.batch_decode(outputs)[0]
print(text)

C:\Users\jonas\AppData\Roaming\Python\Python39\site-packages\transformers\generation\utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\jonas\AppData\Roaming\Python\Python39\site-packages\transformers\generation\utils.py:1262: UserWarning: Input length of input_ids is 570, but `max_length` is set to 128. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Below is a "USER" asking to make a change to a smart home device. Respond in JSON format as defined under "JSON STRUCTURE". All available devices and their alias are defined in the "DEVICE ALIAS LIST" in the format "Alias: device_id", choose the best fitting device. Also choose an "action" from the "ACTION LIST". 


JSON STRUCTURE: {{"type": "command", "commands": [{{"device": device_id, "action": string, "state": string}}]}}

ACTION LIST: power [STATE: on, off], dim [STATE: percentage], color [STATE: color name]

DEVICE LIST: 
bathroom light: light.bathroom,
livingroom light: light.livingroom,
Kitchen light: light.kitchen,
bedroom light: light.bedroom,
garage light: light.garage,
study room light: light.study_room

USER REQUEST: "Turn on the Bathroom Light."
RESPONSE: {{"type": "command", "commands": [{{"device": "light.bathroom", "action": "power", "state": "on"}}]}}

USER REQUEST: "Turn the Livingroom Light on."
RESPONSE: {{"type": "command", "commands": [{{"device": "light.livingro

In [57]:
import langchain
from langchain.llms import HuggingFaceTextGenInference
from langchain import PromptTemplate
from langchain.chains import LLMChain

inference_server_url = 'http://localhost:8080'
llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    #stop_sequences=["USER REQUEST", "USER", "\n\n"],
    max_new_tokens=128,
    #top_k=10,
    top_p=0.90,
    #typical_p=0.95,
    temperature=0.2,
    repetition_penalty=1.2,
)

prompt = PromptTemplate(
    input_variables=["input"],
    template="""{input}"""
)
langchain.debug = True

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("""Below is a \"USER\" asking to make a change to a smart home device. Respond in JSON format as defined under \"JSON STRUCTURE\". All available devices and their alias are defined in the \"DEVICE ALIAS LIST\" in the format \"Alias: device_id\", choose the best fitting device. Also choose an \"action\" from the \"ACTION LIST\". \n\n\nJSON STRUCTURE: {{\"type\": \"command\", \"commands\": [{{\"device\": device_id, \"action\": string, \"state\": string}}]}}\n\nACTION LIST: power [STATE: on, off], dim [STATE: percentage], color [STATE: color name]\n\nDEVICE LIST: \nbathroom light: light.bathroom,\nlivingroom light: light.livingroom,\nKitchen light: light.kitchen,\nbedroom light: light.bedroom,\ngarage light: light.garage,\nstudy room light: light.study_room\n\nUSER REQUEST: \"Turn on the Bathroom Light.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.bathroom\", \"action\": \"power\", \"state\": \"on\"}}]}}\n\nUSER REQUEST: \"Turn the Livingroom Light on.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.livingroom\", \"action\": \"power\", \"state\": \"on\"}}]}}\n\nUSER REQUEST: \"Turn off the Light in the Kitchen.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.kitchen\", \"action\": \"power\", \"state\": \"off\"}}]}}\n\nUSER REQUEST: \"Set the color in the livingroom to green.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.livingroom\", \"action\": \"color\", \"state\": \"green\"}}]}}\n\nUSER REQUEST: \"Enable the Livingroom Light.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [[{{\"device\": \"light.livingroom\", \"action\": \"power\", \"state\": \"on\"}}]}}\n\nUSER REQUEST: \"Dim the light in the kitchen to 50%.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.kitchen\", \"action\": \"dim\", \"state\": \"50\"}}]}}\n\nUSER: \"Turn on the livingroom light.\"}\n\n ASSISTANT:"""))


[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "input": "Below is a \"USER\" asking to make a change to a smart home device. Respond in JSON format as defined under \"JSON STRUCTURE\". All available devices and their alias are defined in the \"DEVICE ALIAS LIST\" in the format \"Alias: device_id\", choose the best fitting device. Also choose an \"action\" from the \"ACTION LIST\". \n\n\nJSON STRUCTURE: {{\"type\": \"command\", \"commands\": [{{\"device\": device_id, \"action\": string, \"state\": string}}]}}\n\nACTION LIST: power [STATE: on, off], dim [STATE: percentage], color [STATE: color name]\n\nDEVICE LIST: \nbathroom light: light.bathroom,\nlivingroom light: light.livingroom,\nKitchen light: light.kitchen,\nbedroom light: light.bedroom,\ngarage light: light.garage,\nstudy room light: light.study_room\n\nUSER REQUEST: \"Turn on the Bathroom Light.\"\nRESPONSE: {{\"type\": \"command\", \"commands\": [{{\"device\": \"light.bathroom\", \"action\": \"power\", \"s

# OLD CODE

Router v2:

In [ ]:
prompt = PromptTemplate(
    input_variables=["User_request", "json_response_template"],
    template="""
### Instruction:    
Below are some "USER REQUESTS" and a "RESPONSE" from a Smart Home System. Respond in a JSON format as defined under "JSON STRUCTURE". If the user is trying to change the state of a device you should change the "REQUEST_TYPE" to be "COMMAND". If the user is requesting information about a device the "REQUEST_TYPE" should be "INQUIRY".

JSON STRUCTURE: {json_response_template}

USER REQUEST: "set the lights in the office to 50%"
RESPONSE: {{"REQUEST_TYPE": "COMMAND",
"response_msg": "Dimming the light now."}}

USER REQUEST: "Turn on the light in the Hallway!"
RESPONSE: {{"REQUEST_TYPE": "COMMAND",
"response_msg": "Turning on the Light."}}

USER REQUEST: "Is the light in the Kitchen on?"
RESPONSE: {{"REQUEST_TYPE": "INQUIRY",
"response_msg": "i will check."}}

USER REQUEST: "What color is the light in the kitchen?"
RESPONSE: {{"REQUEST_TYPE": "INQUIRY",
"response_msg": "let me see."}}

USER REQUEST: "{User_request}"
### RESPONSE: 
"""
)

chain = LLMChain(llm=llm, prompt=prompt)

Router v1:

In [7]:
prompt = PromptTemplate(
    input_variables=["User_request"],
    template="""
### Instruction:    
Below you will find some "USER REQUESTS" and a "RESPONSE" from a Smart Home System. You should respond in a YAML format as defined under "YAML STRUCTURE". If the user is trying to change the state of a device you should change "[X]" to "COMMAND". If the user is requesting information about a device change "[X]" to "INQUIRY".

YAML STRUCTURE: "REQUEST_TYPE: [X]"

USER REQUEST: "set the lights in the office to 50%"
RESPONSE: "REQUEST_TYPE: COMMAND"

USER REQUEST: "Turn on the light in the Hallway!"
RESPONSE: "REQUEST_TYPE: COMMAND"

USER REQUEST: "Is the light in the Kitchen on?"
RESPONSE: "REQUEST_TYPE: INQUIRY"

USER REQUEST: "What color is the light in the kitchen?"
RESPONSE: "REQUEST_TYPE: INQUIRY"

USER REQUEST: "{User_request}"
### RESPONSE: 
"""
)

chain = LLMChain(llm=llm, prompt=prompt)